In [30]:
import scrapfly_scrapers.crunchbase
from dotenv import load_dotenv
from core import init_requests_cache

# config
init_requests_cache()
load_dotenv()

import scrapfly_scrapers.crunchbase.BASE_CONFIG["cache"] = True

In [31]:
from core import CompanyProduct

target = CompanyProduct.same("98point6")

In [ ]:
import scrapfly_scrapers.crunchbase
scrapfly_scrapers.crunchbase.BASE_CONFIG["cache"] = True

crunchbase_response = await crunchbase.scrape_company(url)

pprint(crunchbase_response)

In [20]:
from src.crunchbase.models import *

organization, employees = parse(crunchbase_response)

In [21]:
import jinja2


# Create a Jinja2 environment and load templates from the directory
env = jinja2.Environment(loader=jinja2.FileSystemLoader("templates"))

print(env.get_template("crunchbase.md").render(organization=organization, employees=employees))

# Rad AI, Inc., founded 2018-01-01
Rad AI develops an artificial intelligence automated report generation platform to automate repetitive tasks for radiologists. Its platform integrates with existing workflow, automating repetitive tasks, and enabling radiologists to improve radiology workflow that maximizes efficiency and accuracy while reducing burnout for radiologists and improving patient care.

Website: https://www.radai.com
LinkedIn: https://www.linkedin.com/company/radai
Twitter: https://twitter.com/radai
Facebook: https://m.facebook.com/radaiFB

## Funding (83M USD total)

- 50M USD on 2024-05-07
- 25M USD on 2021-11-10
- 8M USD on 2020-08-20


## News

- Wake Radiology Adds Rad AI Technology to Revolutionize Radiology Reporting (Wake Radiology UNC Rex Healthcare, 2024-07-29)
- Bayer, Rad AI partner for AI radiology workflow offering (Mobi Health News, 2024-07-01)
- Bayer and Rad AI Announce Collaboration agreement (Business Wire, 2024-06-27)
- Most-Active US Investors In May: 

In [29]:
from search import search
from core import CompanyProduct

target = "Rad AI"
# https://www.crunchbase.com/organization/radai/people
# for result in search(f'site:www.crunchbase.com/organization "{target}" people'):
#     pprint(result)

def find_people_url(target: CompanyProduct) -> str:
    result = next(search(f'site:www.crunchbase.com/organization "{target.company}"', num=1))
    assert "/organization/" in result.link

    return f"{result.link}/people"

print(find_people_url(CompanyProduct.same("Pomelo Care")))

https://www.crunchbase.com/organization/pomelo-care/people
